## Check Member End Forces
This notebook reads files describing a structure, and the files output by Frame2D after an
analysis, and checks that the member end forces computed here from the displacements and
member loads agree with those computed by Frame2D.

It does this in the simplest way possible, using quite different logic than Frame2D, resulting
in a higher degree of confidence in the results.  It would have been better had someone else
programmed it, but oh well ...

In [33]:
dir = 'KG82'
#dir = 'l22x6'
#dir = 'l22x6pd'

def filename(basename):
    return dir + '.d/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [34]:
import pandas as pd
import math

In [35]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.deltaX = 0.
        self.deltaY = 0.
        self.thetaZ = 0.

In [36]:
ntable = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in ntable.iterrows():
    if n.ID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.ID))
    NODES[n.ID] = Node(n.ID,float(n.X),float(n.Y))

In [37]:
dtable = pd.read_csv(filename('displacements'))
for i,n in dtable.iterrows():
    node = NODES[n.ID]
    node.deltaX = float(n.DX)
    node.deltaY = float(n.DY)
    node.thetaZ = float(n.RZ)

In [38]:
pd.DataFrame([vars(v) for v in NODES.values()]).set_index('id')

,deltaX,deltaY,thetaZ,x,y
id,,,,,
B,22.180393,-2.628151,-0.007170,10000,6500
K,22.164771,-1.321138,-0.002797,0,6500
J,0.000000,0.000000,-0.003717,0,0
G,22.674076,-1.321138,-0.002756,30500,6500
D,0.000000,0.000000,0.000000,20500,0
I,0.000000,0.000000,-0.003855,30500,0
C,34.684582,-3.639574,-0.007412,10000,12000
A,0.000000,0.000000,0.000000,10000,0
L,34.693044,-1.824187,-0.002018,0,12000


In [39]:
dtable

,ID,DX,DY,RZ
0,A,0.000000,0.000000,0.000000
1,B,22.180393,-2.628151,-0.007170
2,C,34.684582,-3.639574,-0.007412
3,D,0.000000,0.000000,0.000000
4,E,22.531994,-2.788516,0.003096
5,F,34.332031,-3.839593,0.005704
6,G,22.674076,-1.321138,-0.002756
7,H,34.423249,-1.824187,-0.001826
8,I,0.000000,0.000000,-0.003855
9,J,0.000000,0.000000,-0.003717


In [40]:
class Member(object):
    
    E = 200000.
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L
        
        self.Ix = 0.
        self.A = 0.
        self.loads = []
        self.releases = set()
        
        for a in 'FXJ FXK FYJ FYK MZJ MZK'.split():
            setattr(self,a,0.)

In [41]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.ID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.ID))
    MEMBERS[m.ID] = Member(m.ID,NODES[m.NODEJ],NODES[m.NODEK])

In [42]:
import sst
SST = sst.SST()
table = pd.read_csv(filename('properties'))
defIx = defA = None
for i,row in table.iterrows():
    if not pd.isnull(row.SIZE):
        defIx,defA = SST.section(row.SIZE,'Ix,A')
    memb = MEMBERS[row.ID]
    memb.Ix = float(defIx if pd.isnull(row.Ix) else row.Ix)
    memb.A = float(defA if pd.isnull(row.A) else row.A)
    if not pd.isnull(row.Ix):
        defIx = row.Ix
    if not pd.isnull(row.A):
        defA = row.A

In [43]:
table = pd.read_csv(filename('member_loads'))
for i,row in table.iterrows():
    memb = MEMBERS[row.ID]
    typ = row.TYPE
    w1 = None if pd.isnull(row.W1) else float(row.W1)
    w2 = None if pd.isnull(row.W2) else float(row.W2)
    a = None if pd.isnull(row.A) else float(row.A)
    b = None if pd.isnull(row.B) else float(row.B)
    c = None if pd.isnull(row.C) else float(row.C)
    memb.loads.append((typ,w1,w2,a,b,c))

In [44]:
table = pd.read_csv(filename('releases'))
for i,row in table.iterrows():
    memb = MEMBERS[row.ID]
    memb.releases.add(row.R.upper())

In [45]:
t = pd.DataFrame([vars(v) for v in MEMBERS.values()]).set_index('id')
del t['nodej']
del t['nodek']
del t['loads']
t

,A,FXJ,FXK,FYJ,FYK,Ix,L,MZJ,MZK,cosx,cosy,releases
id,,,,,,,,,,,,
IG,12300,0,0,0,0,222000000,6500,0,0,0,1,set([])
KB,13500,0,0,0,0,488000000,10000,0,0,1,0,"set([MZK, MZJ])"
DE,12300,0,0,0,0,222000000,6500,0,0,0,1,set([])
FH,13500,0,0,0,0,488000000,10000,0,0,1,0,"set([MZK, MZJ])"
GH,12300,0,0,0,0,222000000,5500,0,0,0,1,set([])
AB,12300,0,0,0,0,222000000,6500,0,0,0,1,set([])
BE,13500,0,0,0,0,488000000,10500,0,0,1,0,set([])
KL,12300,0,0,0,0,222000000,5500,0,0,0,1,set([])
BC,12300,0,0,0,0,222000000,5500,0,0,0,1,set([])


In [46]:
MEFS = pd.read_csv(filename('mefs')).set_index('ID')
MEFS

,FXJ,FYJ,MZJ,FXK,FYK,MZK
ID,,,,,,
AB,994654.015809,-2177.821451,4.189959e+07,-994654.015809,2177.821451,-5.605543e+07
BC,452381.814808,-88371.309081,-2.410726e+08,-452381.814808,88371.309081,-2.449696e+08
DE,1055345.984190,63235.567017,1.843676e+08,-1055345.984190,-63235.567017,2.266636e+08
EF,470118.185192,115284.882504,2.959813e+08,-470118.185192,-115284.882504,3.380856e+08
IG,500000.000000,2308.981361,3.725290e-08,-500000.000000,-2308.981361,1.500838e+07
GH,225000.000000,-2728.796153,-1.500838e+07,-225000.000000,2728.796153,1.490116e-08
JK,500000.000000,1933.273074,0.000000e+00,-500000.000000,-1933.273074,1.256627e+07
KL,225000.000000,-2284.777269,-1.256627e+07,-225000.000000,2284.777269,0.000000e+00
CF,90656.086350,227381.814808,2.449696e+08,-90656.086350,245118.185192,-3.380856e+08


In [55]:
cols = 'FXJ FXK FYJ FYK MZJ MZK'.split()
for m in MEMBERS.values():
    for a in cols:
        setattr(m,a,0.)
    dX = m.nodek.deltaX - m.nodej.deltaX
    dY = m.nodek.deltaY - m.nodej.deltaY
    # axial deformation / force:
    ldX = dX*m.cosx + dY*m.cosy
    T = m.E*m.A*ldX/m.L
    m.FXK += T
    m.FXJ += -T
    #print(m.id,ldX,T)
    # shear deformation / force:
    vdY = dY*m.cosx - dX*m.cosy
    M = -6.*m.E*m.Ix*vdY/(m.L*m.L)
    V = -2.*M/m.L
    m.MZJ += M
    m.MZK += M
    m.FYJ += V
    m.FYK += -V
    print(m.id,vdY,M,V)
    # end rotations / moments:
    MJ = (m.E*m.Ix/m.L)*(4.*m.nodej.thetaZ + 2.*m.nodek.thetaZ)
    MK = (m.E*m.Ix/m.L)*(2.*m.nodej.thetaZ + 4.*m.nodek.thetaZ)
    VJ = -(MJ+MK)/m.L
    m.MZJ += MJ
    m.MZK += MK
    m.FYJ += VJ
    m.FYK += -VJ
    #print(m.id,m.nodej.thetaZ,m.nodek.thetaZ,MJ,MK,VJ)

IG -22.6740760496 142967428.6299039 -43989.97803997043
KB -1.3070126434 7653866.0397504 -1530.77320795008
DE -22.5319937441 142071553.45392284 -43714.32413966856
FH 2.01540606333 -11802217.90686048 2360.443581372096
GH -11.749172545700002 103470398.88180101 -37625.59959338218
AB -22.1803933186 139854598.34497136 -43032.18410614503
BE -0.1603649571100001 851788.8334114836 -162.24549207837782
KL -12.528273506699996 110331638.41933484 -40120.59578884904
BC -12.504188960099999 110119535.17258312 -40043.46733548477
EG 1.46737760051 -8592963.22858656 1718.592645717312
EF -11.800037087699998 103918343.14589354 -37788.48841668856
LC -1.8153866195899995 10630904.044319037 -2126.1808088638077
CF -0.20001944374000047 1062416.2018516487 -202.36499082888548
JK -22.16477091 139756093.97453254 -43001.875069086935


In [56]:
computed = pd.DataFrame([{k:getattr(m,k) for k in ['id']+cols} 
                         for m in MEMBERS.values()]).set_index('id')
(computed - MEFS[cols])

,FXJ,FXK,FYJ,FYK,MZJ,MZK
AB,4.610047e-08,-4.610047e-08,4355.642903,-4355.642903,-2.837330e-04,-3.388152e-04
BC,-2.036570e-06,2.036570e-06,176742.618162,-176742.618162,1.241267e-04,7.423460e-04
BE,-1.291405e-05,1.291405e-05,-245794.402002,-331705.597998,-5.053125e+08,5.053125e+08
CF,-4.485322e-06,4.485322e-06,-218513.629616,-253986.370384,-4.134375e+08,4.134375e+08
DE,2.215384e-06,-2.215384e-06,-126471.134034,126471.134034,2.500415e-05,1.271069e-04
EF,-2.436398e-06,2.436398e-06,-230569.765008,230569.765008,-7.659197e-05,5.910397e-04
EG,1.099819e-06,-1.099819e-06,-275272.824355,-274727.175645,5.847888e+07,-5.575063e+07
FH,8.398685e-06,-8.398685e-06,-245345.895251,-204654.104749,1.752238e+08,2.823512e+07
GH,9.819050e-07,-9.819050e-07,5457.592307,-5457.592307,1.184605e-04,2.935529e-05
IG,-8.000643e-07,8.000643e-07,-4617.962721,4617.962721,2.521798e-04,2.500601e-04
